In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras.models
from keras.layers.convolutional import Conv1D, Conv2D, ZeroPadding1D, ZeroPadding2D
from keras.layers import Embedding
from keras.utils import plot_model
from sklearn.preprocessing import LabelBinarizer
from keras.utils import np_utils
from math import floor, ceil
from sklearn.metrics import accuracy_score

def load_and_format_data():
    # load data
    dataset = 'train/1'
    x_train = pd.read_csv(dataset + '.train.calcium.csv')['0']  # todo: concat these
    y_train = pd.read_csv(dataset + '.train.spikes.csv')['0']
    x_test = pd.read_csv(dataset + '.train.calcium.csv')['1']  # todo: concat these
    y_test = pd.read_csv(dataset + '.train.spikes.csv')['1']
    # binarize labels
    lb = LabelBinarizer()
    lb.fit(y_train) # this lb takes 0 -> [1,0,0,0,0], 1-> [0,1,0,0,0], etc.
    y_train = lb.transform(y_train)
#     y_test = lb.transform(y_test)
    # reshape data
    x_train = x_train.values.reshape((1, len(x_train), 1))
    y_train = y_train.reshape((1, len(y_train), y_train.shape[1]))
    x_test = x_test.values.reshape((1, len(x_test), 1))
#     y_test = y_test.reshape((1, len(y_test), y_test.shape[1]))
    return x_train, y_train, x_test, y_test


def plot(calcium, spikes, xlim):
    t = np.arange(len(calcium)) / 100.0
    plt.figure(figsize=(15, 3))
    plt.plot(t, calcium, color=(.1, .6, .4))
    plt.plot(t, spikes / 2.0 - 1, color='k')
    plt.yticks([])
    plt.ylim([-2., 4.])
    plt.xlim(xlim)
    plt.grid()
    plt.show()

# prepare inputs

In [56]:
x_train, y_train, x_test, y_test = load_and_format_data()

# create model

In [32]:
max_features = 10
embedding_dims = 10
maxlen = 100
kernel_size = 5
pad_size = floor(kernel_size / 2)
model = keras.models.Sequential()
model.add(ZeroPadding1D(padding=pad_size, input_shape=(None, 1)))
model.add(Conv1D(filters=5, kernel_size=kernel_size, activation="softmax"))
# model.add(Dense(12, input_dim=8, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# compile model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# fit the model

In [41]:
epochs = 100
model.fit(x_train, y_train, epochs=epochs, verbose=0)

# evaluate

In [63]:
print('predict...')
y_pred = model.predict(x_test)[0,:,:]
print(y_pred)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)
print(y_test.shape)
print('y_pred nonzeros',str(np.sum(y_pred) / y_pred.size))
print("acc",accuracy_score(y_pred,y_test))

# evaluate the model
print('evaluating...')

predict...
[[ 0.26623961  0.11127386  0.21197119  0.26301327  0.14750203]
 [ 0.29660207  0.06754951  0.20933472  0.2900295   0.13648428]
 [ 0.37587559  0.06536113  0.17672665  0.26159251  0.12044407]
 ..., 
 [ 0.29707941  0.11918648  0.19793138  0.22511289  0.16068982]
 [ 0.325609    0.14013241  0.17212558  0.21240133  0.14973164]
 [ 0.3196809   0.13500638  0.16950004  0.19854881  0.17726389]]
(71986,)
(71986,)
y_pred nonzeros 0.0778206873559
acc 0.952393520962
evaluating...
